# Analyzing the High-Risk Area

### Connect your GIS (optional)

In [1]:
# The results generated in this notebook will be shared as a Web Map in my ArcGIS Online account, and I will import them into this notebook.
from arcgis.gis import GIS
gis = GIS('home')

### Input data and the source

Digital Elevation Model (DEM): NASA SRTM  
Slope: NASA SRTM  
Land Use and Land Cover: European Space Agency (ESA)  
Precipitation: WorldClim  
Distance to streams: Accumulated from DEM  
Boundary: National Land Surveying and Mapping Center (Taiwan)  

----------------------------------------------------------------------------------------------------

In [19]:
from arcgis.gis import GIS
from arcgis.mapping import WebMap

The code down below is to utilize DEM data to obtain distance to streams:

**Fill Sinks**. This process helps in creating a continuous and accurate surface model

In [ ]:
with arcpy.EnvManager(scratchWorkspace=r"D:\fall2023\arc1\project\Flood\Flood.gdb"):
    out_surface_raster = arcpy.sa.Fill(
        in_surface_raster="DEM_export_Clip",
        z_limit=None
    )
    out_surface_raster.save(r"D:\fall2023\arc1\project\Flood\Flood.gdb\Fill")

**Flow Direction (D8)**. D8 method are typically defined as the eight immediately adjacent cells: north, northeast, east, southeast, south, southwest, west, and northwest. The direction in which water flows from a particular cell is determined by identifying the steepest downward slope among these neighboring cells.

In [ ]:
with arcpy.EnvManager(scratchWorkspace=r"D:\fall2023\arc1\project\Flood\Flood.gdb"):
    out_flow_direction_raster = arcpy.sa.FlowDirection(
        in_surface_raster="Fill",
        force_flow="NORMAL",
        out_drop_raster=None,
        flow_direction_type="D8"
    )
    out_flow_direction_raster.save(r"D:\fall2023\arc1\project\Flood\Flood.gdb\FlowDirection")

**Flow Accumulation**. This information is crucial for understanding the distribution and concentration of water flow in a landscape.

In [ ]:
with arcpy.EnvManager(scratchWorkspace=r"D:\fall2023\arc1\project\Flood\Flood.gdb"):
    out_accumulation_raster = arcpy.sa.FlowAccumulation(
        in_flow_direction_raster="FlowDirection",
        in_weight_raster=None,
        data_type="FLOAT",
        flow_direction_type="D8"
    )
    out_accumulation_raster.save(r"D:\fall2023\arc1\project\Flood\Flood.gdb\FlowAcc")

**Reclassify**. If the value is greater than a threshold ( 1% of the maximum flow accumlation ), assign it to 1; otherwise, assing it to NODATA.

In [ ]:
with arcpy.EnvManager(scratchWorkspace=r"D:\fall2023\arc1\project\Flood\Flood.gdb"):
    out_raster = arcpy.sa.Reclassify(
        in_raster="FlowAcc",
        reclass_field="VALUE",
        remap="0 76000 NODATA;76000 7603437 1",
        missing_values="NODATA"
    )
    out_raster.save(r"D:\fall2023\arc1\project\Flood\Flood.gdb\streams")

**Euclidean Distance**. Distance to streams.

In [ ]:
with arcpy.EnvManager(scratchWorkspace=r"D:\fall2023\arc1\project\Flood\Flood.gdb"):
    out_distance_raster = arcpy.sa.EucDistance(
        in_source_data="streams",
        maximum_distance=None,
        cell_size=r"D:\fall2023\arc1\project\Flood\DEM_export.tif",
        out_direction_raster=None,
        distance_method="PLANAR",
        in_barrier_data=None,
        out_back_direction_raster=None
    )
    out_distance_raster.save(r"D:\fall2023\arc1\project\Flood\Flood.gdb\Dist_streams")

### Assign the weights for input data
Reclassify each of the input data into five classes

In [ ]:
# DEM
with arcpy.EnvManager(extent='114.359282472295 10.3713476630824 124.561149500316 26.3852781295282 GEOGCS["GCS_WGS_1984",DATUM["D_WGS_1984",SPHEROID["WGS_1984",6378137.0,298.257223563]],PRIMEM["Greenwich",0.0],UNIT["Degree",0.0174532925199433]]', scratchWorkspace=r"D:\fall2023\arc1\project\Flood\Flood.gdb"):
    out_raster = arcpy.sa.Reclassify(
        in_raster="DEM_export.tif",
        reclass_field="Value",
        remap="-45 234.670588 10;234.670588 825.086275 8;825.086275 1524.262745 6;1524.262745 2285.588235 4;2285.588235 3917 2",
        missing_values="NODATA"
    )
    out_raster.save(r"D:\fall2023\arc1\project\Flood\Flood.gdb\Reclass_DEM")

In [ ]:
# Slope
with arcpy.EnvManager(scratchWorkspace=r"D:\fall2023\arc1\project\Flood\Flood.gdb"):
    out_raster = arcpy.sa.Reclassify(
        in_raster="Slope_export.tif",
        reclass_field="VALUE",
        remap="0 2 10;2 5 8;5 10 6;10 20 4;20 78.986473 2",
        missing_values="NODATA"
    )
    out_raster.save(r"D:\fall2023\arc1\project\Flood\Flood.gdb\Reclass_Slop")

In [ ]:
# Precipitaiton
with arcpy.EnvManager(scratchWorkspace=r"D:\fall2023\arc1\project\Flood\Flood.gdb"):
    out_raster = arcpy.sa.Reclassify(
        in_raster="prc",
        reclass_field="VALUE",
        remap="88 266 2;266 370 4;370 497 6;497 655 8;655 1093 10",
        missing_values="NODATA"
    )
    out_raster.save(r"D:\fall2023\arc1\project\Flood\Flood.gdb\Reclass_prc")

In [ ]:
#LULC
with arcpy.EnvManager(scratchWorkspace=r"D:\fall2023\arc1\project\Flood\Flood.gdb"):
    out_raster = arcpy.sa.Reclassify(
        in_raster="ESA_LULC_ROI.tif",
        reclass_field="Value",
        remap="0 NODATA;10 2;20 4;30 4;40 6;50 8;60 8;80 10;90 8;95 8",
        missing_values="NODATA"
    )
    out_raster.save(r"D:\fall2023\arc1\project\Flood\Flood.gdb\Reclass_ESA")

In [ ]:
#Distance to streams
with arcpy.EnvManager(scratchWorkspace=r"D:\fall2023\arc1\project\Flood\Flood.gdb"):
    out_raster = arcpy.sa.Reclassify(
        in_raster="Distanc_stre1",
        reclass_field="VALUE",
        remap="0 0.014320 10;0.014320 0.031027 8;0.031027 0.049524 6;0.049524 0.072795 4;0.072795 0.152153 2",
        missing_values="NODATA"
    )
    out_raster.save(r"D:\fall2023\arc1\project\Flood\Flood.gdb\Reclass_Dist")

Employ the Analytic Hierarchy Process (AHP) to determine the probability weights. AHP method was developed by Thomas L. Saaty in the 1980s

In [5]:
import numpy as np

def ahp(matrix):
    # Normalizing the matrix
    normalized_matrix = matrix / matrix.sum(axis=0)

    # Calculating the weights
    weights = normalized_matrix.mean(axis=1)

    # Normalizing the weights
    normalized_weights = weights / weights.sum()

    return normalized_weights

# Example usage
criteria_matrix = np.array([
    [1, 1/2, 1/4, 1/3, 1/7],
    [2, 1, 1/5, 1/2, 1/6],
    [4, 5, 1, 5, 1/3],
    [3, 2, 1/5, 1, 1/5],
    [7, 6, 3, 5, 1]
])

criteria_weights = ahp(criteria_matrix)

# Print the results
criteria = ['Slope', 'DEM', 'Distance to river', 'LULC', 'Precipitation']
for i in range(len(criteria)):
    print(f"{criteria[i]} Weight: {criteria_weights[i]}")


Slope Weight: 0.050551624564427536
DEM Weight: 0.0724632252731259
Distance to river Weight: 0.2797178463067995
LULC Weight: 0.11008930988853834
Precipitation Weight: 0.48717799396710876


In [ ]:
with arcpy.EnvManager(scratchWorkspace=r"D:\fall2023\arc1\project\Flood\Flood.gdb"):
    out_raster = arcpy.sa.WeightedSum(
        in_weighted_sum_table="Reclass_Dist Value 0.278;Reclass_ESA Value 0.103;Reclass_prc Value 0.5;Reclass_Slop Value 0.049;Reclass_DEM Value 0.07"
    )
    out_raster.save(r"D:\fall2023\arc1\project\Flood\Flood.gdb\Weight")